# CS1470/2470 HW4: Language Models

In this homework assignment, you will build deep learning language models. 

---

In [1]:
import numpy as np
import tensorflow as tf

import matplotlib as mpl
import matplotlib.pyplot as plt
import preprocessing
import rnn

print(tf.__version__)
print(np.__version__)

import collections

2.6.0
1.19.2


In [2]:
%load_ext autoreload
%autoreload 2
# import   preprocess, trigram, rnn
#%aimport preprocessing.py#, trigram, rnn

In [3]:
data_path = "../data" ## TODO: Maybe edit if need be?

## Preprocess

### Tokenized Corpus

It is now the time for you to finish the `get_data` function in the file `preprocess.py`. Then come back to this notebook and run the following cell.

In [4]:
# ## get the tokenized list of words from the corpus
# train_words_tokenized, test_words_tokenized, word_to_token_dict, relevance_dictionary = preprocessing.preprocess(directory='../data_info/data/', train_test_split=0.8, k=2)

# ## A useful utility for counting things
# word_counter = collections.Counter(train_words_tokenized)

# ## What are the 40 most common words?
# n_most_common = 40
# most_common_tokens, most_common_occurrences = zip(*word_counter.most_common(n_most_common))

# ## Convert the tokens back to words so that we can see what they are
# token_to_word_dict = {i:w for w, i in word_to_token_dict.items()}
# most_common_words = [token_to_word_dict[t] for t in most_common_tokens]

# print(*zip(most_common_words, most_common_occurrences), sep = ", ")

Here is a histogram for you. Notice how the number of occurrences decreases exponentially. 

In [5]:
# fig_most_common, ax_top50_most_common = plt.subplots()
# ax_top50_most_common.barh(y = most_common_words,
#                           width = most_common_occurrences, 
#                           height = 0.75, 
#                           color = "C0", 
#                           edgecolor = "black", 
#                           zorder = 100)

# ax_top50_most_common.grid(linestyle = "dashed", 
#                           color = "#bfbfbf", 
#                           zorder = -100)

# ax_top50_most_common.set_yticks(ticks = ax_top50_most_common.get_yticks())
# ax_top50_most_common.set_yticklabels(labels = most_common_words, 
#                                      fontsize = 14)

# ax_top50_most_common.invert_yaxis()
# ## If you want log-scale 
# # ax_top50_most_common.set_xscale('log')

# fig_most_common.set_size_inches([6, 12])

### Train and Test with RNN

Now you should be ready to complete every function in the file `trigram.py`. Go finish `trigram.py` and come back to here.

Steps to take: 
- Load the data with `get_data`
- Reshape the input and output data into the RNN shape
- Initialize the model, train it, and calculate the perplexity

In [20]:
%%time

import rnn

train_id, test_id, vocab, relevance, lp = preprocessing.preprocess(directory='../data_info/data/', train_test_split=0.8, k=3)

train_id = np.array(train_id)
test_id  = np.array(test_id)    

# Training and validation are aligned because we require the input song for RPrecision
X0, Y0 = train_id, train_id
X1, Y1 = test_id,  test_id

## TODO: Get your model that you'd like to use
args = rnn.get_text_model(vocab, relevance)

data = args.model.fit(
    X0, Y0,
    epochs=2, 
    batch_size=lp,
    validation_data=(X1, Y1)
)

Epoch 1/2


ResourceExhaustedError: failed to allocate memory [Op:Mul]

### Generate Sentences with RNN

Try the model with your own pairs of starting words. 

In [19]:
def RPrecision(predictions, labels):
        PAD_TOKEN = 0
        #print(prediction_arr)
        predict_set = set(predictions)
        labels = labels[:len(predict_set)]
        
        ground_truth = set(labels)

        # Return mean of running total to get running mean
        return len(predict_set.intersection(ground_truth)) / len(ground_truth)

## Feel free to mess around with the word list to see the model try to generate sentences
for word1 in ['Closer']:
    if word1 not in vocab: print(f"{word1} not in vocabulary")            
    else: print(args.model.generate_recommendations(word1, 10, vocab))
    print()

ids = relevance[vocab['Closer']]
id_to_track = {id: name for name, id in vocab.items()}
tracks =[id_to_track[id] for id in ids]
print(tracks[:30])
print()
print("R-Precision: " + str(RPrecision(args.model.generate_recommendations(word1, 10, vocab), [id_to_track[x] for x in relevance[vocab['Closer']]])))

['Alone', 'Let It Go', 'Stay With Me', "Can't Hold Us - feat. Ray Dalton", 'Unforgettable', 'New Americana', 'Skinny Love', 'Magnolia', 'Sorry Not Sorry', 'Closer']

['Closer', 'Let Me Love You', 'One Dance', 'Cold Water (feat. Justin Bieber & MØ)', 'Broccoli (feat. Lil Yachty)', 'Roses', "Don't Let Me Down", 'Sorry', 'Starboy', 'This Is What You Came For', 'Work from Home', 'I Took A Pill In Ibiza - Seeb Remix', 'Gold', 'Never Be Like You', 'Panda', 'Shape of You', 'Trap Queen', 'Paris', 'Ride', 'No Problem (feat. Lil Wayne & 2 Chainz)', 'CAN\'T STOP THE FEELING! (Original Song from DreamWorks Animation\'s "TROLLS")', 'Heathens', 'Black Beatles', 'Stay', 'Jumpman', 'The Hills', 'Starving', 'In the Name of Love', 'Work', 'Hands To Myself']

R-Precision: 0.1
